In [1]:
# third party
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pathlib

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# syft absolute
import syft as sy

2024-01-19 02:40:56.333346: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-19 02:40:56.352457: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-19 02:40:56.352473: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-19 02:40:56.352487: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-19 02:40:56.356206: I tensorflow/core/platform/cpu_feature_g

In [2]:
# Sign in with a Data Scientist account provided by Server Owner
sy.requires(">=0.8.3,<0.8.4")
client = sy.login(port=3000, email=input('Enter Email'), password=input('Enter Password'))

✅ The installed version of syft==0.8.4b12 matches the requirement >=0.8.3 and the requirement <0.8.4
Logged into <remedy: High side Domain> as <kenjiw@davidsonacademy.unr.edu>


In [3]:
datasets = client.datasets.get_all()
datasets

DictTuple(syft.service.dataset.dataset.Dataset,)

In [4]:
all_pointers = {}
for asset in datasets[0].assets:
	all_pointers[asset.name] = asset.pointer

In [5]:
@sy.syft_function(
    input_policy=sy.ExactMatch(
        **all_pointers
    ),
    output_policy=sy.SingleExecutionExactOutput(),
)
def image_classification_model(**data):
	import matplotlib.pyplot as plt
	import tensorflow as tf
	import numpy as np
	
	from tensorflow import keras
	from tensorflow.keras import layers
	from tensorflow.keras.models import Sequential
	
	image_data = []
	label_data = []
	
	for (i, image_array) in enumerate(data.values()):
		for o in range(len(image_array)):
			label_data.append(i)
			image_data.append(image_array[o])

	train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, horizontal_flip = True)
	train_dataset = train_generator.flow(
		x=np.array(image_data),
		y=label_data,
		batch_size=32
	)

	train_dataset.class_names = list(data.keys())

	num_classes = len(train_dataset.class_names)
	model = Sequential([
		layers.RandomRotation(0.1, batch_input_shape=(32,256,256,3)),
		layers.RandomZoom(0.1),
		layers.Conv2D(256, 3, padding='same', activation='relu'),
		layers.MaxPooling2D(),
		layers.Conv2D(128, 3, padding='same', activation='relu'),
		layers.MaxPooling2D(),
		layers.Conv2D(64, 3, padding='same', activation='relu'),
		layers.MaxPooling2D(),
		layers.Conv2D(32, 3, padding='same', activation='relu'),
		layers.MaxPooling2D(),
		layers.Flatten(),
		layers.Dense(32, activation='relu'),
		layers.Dense(num_classes)
	])

	model.compile(optimizer='adam',
		loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
		metrics=['accuracy'])

	epochs=10
	history = model.fit(
		train_dataset,
		validation_data=train_dataset,
		epochs=epochs
	)

	model.save('temp.keras')
	f = open('temp.keras', mode="rb")
	result = f.read()

	return result, train_dataset.class_names

SyftSuccess: Syft function 'image_classification_model' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [6]:
initialize_remedy_project = sy.Project(
    name="REMEDY Classifier",
    description='Please run this Image Classifier in your background servers when you have.\n\nIn Gratitude,\nRemedy Team',
    members=[client],
)
initialize_remedy_project

syft.service.project.project.ProjectSubmit

In [7]:
initialize_remedy_project.create_code_request(obj=image_classification_model, client=client)

SyftSuccess: Code request for 'image_classification_model' successfully added to 'REMEDY Classifier' Project. To see code requests by a client, run `[your_client].code`

In [8]:
# Notify to the Hospital to continue their steps
remedy_project = initialize_remedy_project.start()

In [9]:
remedy_project.events

[syft.service.project.project.ProjectRequest]

In [10]:
client.code

In [11]:
# Only run if client code status is approved AKA when the hospitals finish running your code
result_pointer = client.code.image_classification_model()
real_result, class_names = result_pointer.get()
with open('temp.keras', "wb") as file:
	file.write(real_result)
	file.close()

In [12]:
model = keras.models.load_model('temp.keras')

2024-01-19 02:46:02.542547: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 02:46:02.560191: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-19 02:46:02.560322: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [17]:
predictions = model.predict(tf.expand_dims(tf.keras.utils.img_to_array(tf.keras.utils.load_img(pathlib.Path(input('Enter Image File Path To Test: ')), target_size=(256,256))), 0))

1/1 [==============================] - 0s 11ms/step


In [18]:
score = tf.nn.softmax(predictions[0])

In [19]:
score

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)>

In [20]:
print('This image most likely belongs to {}'.format(class_names[np.argmax(score)]))

This image most likely belongs to Melanoma
